# Introduction to Nettlesome

Nettlesome is a Python library that lets you use readable English phrases as semantic tags for your data. Once you've created tags, Nettlesome can automate the process of comparing the tags to determine whether they have the 
*same meaning*, whether one tag *implies* another, whether one tag *contradicts* another, or whether one tag is *consistent with* another.

This tutorial will show you how to use some of Nettlesome's most important classes: Predicates, Statements, and Comparisons.

# Predicates

A Predicate should contain an English-language phrase in the past tense. (The past tense reflect's Nettlesome's original use case of legal analysis, which is usually backward-looking, determining the legal effect of past acts or past conditions.)

To create a useful Predicate, you might remember from grade school that in a simple sentence the "subject" and any "objects" are typically nouns, while the "predicate" is a verb phrase that describes an action or relationship of those nouns. In Nettlesome the "subject" and "objects" are simply called "Terms", and they should be marked up using the placeholder syntax for Template Strings, which are a part of the Python standard library. Generally speaking, objects that are part of your data model should be designated as Terms, while other nouns that aren't relevant enough to be part of your data model don't need to be terms.

For instance, this example might be a good Predicate for a data model about determining whether an individual is an owner or employee of a company:

In [2]:
from nettlesome import Predicate

account_for_company = Predicate("$applicant opened a bank account for $company")

Because `$applicant` and `$company` are marked as placeholders for Terms, you'll be able to add more data about those entities later. But notice that the "bank account" wasn't designated as a Term. That means you won't be able to link any more Statements about the company's specific bank account. That should be okay as long as your data model won't need to include specific details about bank accounts.

Predicates also have a `truth` attribute that can be used to establish that one Predicate `contradicts` another.

In [7]:
no_account_for_company = Predicate("$applicant opened a bank account for $company", truth=False)
str(no_account_for_company)

'it was false that $applicant opened a bank account for $company'

The `truth` attribute will become more significant as we build up to create more complex objects.

## Formatting Predicates

The placeholders you choose always need to `$start_with_a_dollar_sign` and they need to be valid Python identifiers, which means they can't contain spaces. If a placeholder needs to be adjacent to a non-whitespace character, you also need to `${wrap_it_in_curly_braces}`.

Don't use capitalization or end punctuation to signal the beginning or end of the phrase in a Predicate, because the phrase may be used in a context where it's only part of a longer sentence.

As the example below shows, the use of different placeholders doesn't cause Predicates to be considered to have different meanings. 

In [3]:
account_for_partnership = Predicate("$applicant opened a bank account for $partnership")

account_for_company.means(account_for_partnership)

True

If you need to mention the same term more than once in a Predicate,
use the same placeholder for that term each time. If you later create
a Statement object using the same Predicate, you will only include each unique
Term once, in the order they first appear.

In this example, a Predicate's template has two placeholders referring to the identical Term.
Even though the rest of the text is the same, the reuse of the same Term means that the Predicate has a different meaning.

In [4]:
account_for_self = Predicate("$applicant opened a bank account for $applicant")

account_for_self.means(account_for_company)

False

# Statements

To put it simply, a Statement is a Predicate plus the Terms that need to be included to make the Statement a complete phrase.

In [11]:
from nettlesome import Statement, Entity

statement = Statement(predicate=account_for_company, terms=[Entity("Sarah"), Entity("Acme Corporation")])
str(statement)

'the statement that <Sarah> opened a bank account for <Acme Corporation>'

An Entity is a Term representing a person or thing. If you're lucky enough to be able to run effective Named Entity Recognition techniques on your dataset, you probably already have good candidates for the Entity objects that should be included in your Statements. The data model of an Entity in Nettlesome includes just a `name` attribute, an attribute indicating whether the Entity should be considered `generic`, and a `plural` attribute mainly used to determine whether the word "was" after the Entity should be replaced with "were".

In [13]:
not_at_school = Predicate("$group were at school", truth=False)
plural_statement = Statement(not_at_school, terms=Entity("the students", plural=True))
str(plural_statement)

'the statement it was false that <the students> were at school'

In [15]:
singular_statement = Statement(not_at_school, terms=Entity("Lee", plural=False))
str(singular_statement)

'the statement it was false that <Lee> was at school'

The `generic` attribute is more subtle than the `plural` attribute. A Term should be marked as `generic` if it's really being used as a stand-in for a broader category. For instance, in `singular_statement` above, the fact that `<Lee>` is generic indicates that the Statement isn't really about a specific incident when Lee was not at school. Instead, it's more about the concept of not being at school. In Nettlesome, when angle brackets appear around the string representation of an object, that's an indication that the object is generic.

If two Statements have different generic Terms but they're otherwise the same, they're still considered to have the same meaning as one another. That's the case even if one of the Terms is plural while the other is singular.

In [16]:
plural_statement.means(singular_statement)

True